# Rocked Ratchet (release)

## Introduction

Consider overdamped motion in one dimensional periodic potential dependent explicitely on time:

$$ \dot x = f(x,t) + \sqrt{2D}\xi(t),$$

where:

 - $f(x) = -U'(x)$ and the potential is: $$U(x) = -\frac{1}{2\pi}(\sin(2 \pi x)+ \frac{1}{4} \sin(4 \pi x) + x A \sin(\omega t)$$
 - $\xi(t)$ - white Gaussian noise with mean zero and $\langle \xi(t)\xi(s)\rangle=\delta(t-s)$ correlation function
 - $D$ is thermal diffusion $D=kT/\gamma$ (in this case we have $\gamma=1$)



The objective is to calculate the average speed of the molecule in the $v(t) = \lim_{t\to\infty}\langle x(t)/t \rangle$ system. Averaging is over the implementation of the system (trajectories) and over time (note that the system explicitly contains time). One option is to calculate the position of the particle after a long time and calculate $v = \Delta x / \Delta t$ for each particle.

The alternative is to solve the Fokker-Planck equation:

$$ \frac{\partial P}{\partial t} = -\frac{\partial }{\partial x}\left[ f(x,t) P\right] + D \frac{\partial^2 P}{\partial x^2}$$

with periodic boundary condition $P(x)=P(x+L)$, on one period of the system and calculation of speed from the current probability averaged over time (and space):

$$ J(x,t) = f(x,t) P - D \frac{\partial P}{\partial x}$$,

where $v = \langle J(x,t) \rangle_{x,t} L$.

## Problems

1. Implement the Euler-Maruyama scheme [link] (https://el.us.edu.pl/ekonofizyka/index.php/MKZR:Stochastyczne_równania_różniczkowe) for the above stochastic equation for CUDA.

2. Implement a scheme based on finite differences and explicit integration in time solving the Fokker-Planck equation for CUDA.

3. Recreate, for example, Figure 1 from [link] (http://www.physik.uni-augsburg.de/theo1/hanggi/Papers/163.pdf) each method.

## Numerical integration of Langevin equation

The first approach to this problem will be to solve numrically Langevin equation. We will present an algorithm which will be based on numpy module.

In [ ]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
import sympy
import  time 
from ipywidgets import interact, IntSlider


In [ ]:
from sympy.codegen.ast import real, float32, float64
from sympy.codegen.ast import Declaration, Variable, Pointer

var = lambda x,p:sympy.ccode(Declaration(Variable(sympy.Symbol(x), type=p)) )
pvar = lambda x,p:sympy.ccode(Declaration(Pointer(sympy.Symbol(x), type=p)) )


In [ ]:
precision = float64

if precision == float64:
    np_prec = np.float64

if precision == float32:
    np_prec = np.float32


def make_U_f(precision=float32, A=0.5, omega=1):
    x = sympy.Symbol('x')
    t = sympy.Symbol('t')

    k = 2*sympy.pi
    U = -1/k*(sympy.sin(k*x) + 1/4*sympy.sin(2*k*x)) + x*A*sympy.sin(omega*t)

    f = -sympy.diff(U, x, 1) 
    
    U_lamb = sympy.lambdify([x,t ], U, 'numpy')
    f_lamb = sympy.lambdify([x,t ], f, 'numpy')
    
    f_code = sympy.ccode(f,type_aliases={real: precision})


    
    return U_lamb,f_lamb,f_code,var("",precision),pvar("",precision)

omega = 1.0

U, f, f_code,fp,pfp = make_U_f(precision=precision, A=1.5, omega=omega)
x = np.linspace(-1.2,1.2,100)
plt.figure()
plt.plot(x,U(x,t=0))
plt.show()
print(f_code,fp,pfp)

Let us prepare numerical values of parameters for the simulation:

In [ ]:
N = 1280

T = 2*np.pi/omega
n_periods = 261
T_end = n_periods*T # integer time period 
spp = 1000


dt = T/spp
Dyf = 0.1

a = np.sqrt(2*Dyf*dt)
x = np.zeros(N)
dt,T_end

In [ ]:
nsteps = spp*n_periods
nsteps

In [ ]:
cpu_t = time.time()
for i in range(nsteps):
    t = dt*i
    x += f(x,t)*dt + a*np.random.randn(N) 
cpu_t = time.time() - cpu_t

print( cpu_t, (N*nsteps)/cpu_t/1000**2," M iterations/sek" )
print('t=',nsteps*dt,"v =", np.mean(x)/(nsteps*dt), "  Deff =",np.var(x)/(2*nsteps*dt))

Having positions of particles at some given time, we can estimate the probability density function. We will use `histogram` function included in  'numpy' module. Note, that the option `density=True` will return normalized probablility density instead counts in intervals.

In [ ]:
hist_cpu,xs = np.histogram(x, np.linspace(-50,250,301), density=True)
xs = (xs[1:]+xs[:-1])/2

plt.figure()

plt.plot(xs,hist_cpu)
plt.show()

## Numerical solution of Fokker Plank equation

We will numerically solve the Fokker-Plank equation which is an equivalent descriprion of this problem.
For this purpose we will use finite differences on regular grid in space and explicit Euler scheme in time.

In [ ]:
import time
import numpy as np


T = 2*np.pi/omega
n_periods = 3
total_t = n_periods*T # integer time period 

spp = 20000
dt = T/spp


x1,x2 = 0,1

N = 100  # space discretization
h = (x2-x1)/(N-1) 


Nsteps = spp*n_periods


X = np.linspace(x1, x2, N+1)[:-1]
t = np.linspace(0,total_t,Nsteps+1)
dt = t[1] - t[0]

print( "N=",N,"dt=",dt,'Nsteps=',Nsteps)

u = np.ones(N)

tm = time.time()
every = 100

ulst = []
tlst = []
flst = []
for i in range(Nsteps):
  
    F = f(X,i*dt)

    u[1:-1] = u[1:-1] + dt*( -np.gradient(F*u)[1:-1]/h + Dyf/h**2*np.diff(u,2))

    u[0] =  u[0] + dt*(-(F[1]*u[1]-F[-1]*u[-1])/(2*h) + Dyf/h**2*(u[-1]+u[1]-2.0*u[0]) )
    u[-1] =  u[-1] + dt*(-(F[0]*u[0]-F[-2]*u[-2])/(2*h) + Dyf/h**2*(u[-2]+u[0]-2.0*u[-1]) )
    
    if np.nanmax(np.abs(u))>1e6:
        break
    
    if i%every == 0:
        ulst.append(u.copy())
        tlst.append(i*dt)
        flst.append(f(X,i*dt))
        print(i,end='\r')

tm = time.time()-tm
print ("Saved ",len(tlst), " from ", Nsteps, "h= ",h)

print( tm,"s")

## Comparison of results

### Probability density function

We will now compare $P(x,t)$ obtained in above algorithm with estimation of density of particles computed from SDE simulation.

In [ ]:
x_p = np.fmod(x, 1.0)
plt.figure(figsize=(9,4))
hist,xs = np.histogram(x_p, np.linspace(0,1,240), density=True)
xs = (xs[1:]+xs[:-1])/2
plt.plot(xs,hist)
plt.plot(X,u)


### Average velocity in the system

We can compare probability flux $J(x,t)$ with average velocity obtained from SDE simulation.

In [ ]:
Js =  [ np.sum(h*(ft*u - Dyf/h*np.gradient(u) )) for ft,u in zip(flst,ulst)]
Js  = np.array(Js)
print (tm,"s")

In [ ]:
np.mean(Js)

In [ ]:
plt.plot(tlst,Js)

In [ ]:
np.mean(Js),np.polyfit(tlst,np.cumsum(Js)*dt,1)

Values are close, but higher precision is required.

\newpage